In [219]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [220]:
#dados
train=pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test=pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [221]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [222]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [223]:
#tratando os dados
#separando por titulo
for df in [train, test]:
    df["Title"] = df["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip() if ',' in x and '.' in x.split(',')[1] else '')
    df["Title"] = df["Title"].replace(["Don", "Rev", "Dr", "Major", "Lady", "Sir", "Col", "Capt", "the Countess", "Jonkheer"], "Rare")
    df["Title"] = df["Title"].replace(["Mlle", "Ms"], "Miss")
    df["Title"] = df["Title"].replace(["Mme"], "Mrs")

In [224]:
target='Survived'
x_train,x_test,y_train,y_test=train_test_split(train.drop(target,axis=1),train[target],test_size=.3,random_state=0)

df_train=pd.concat([x_train,y_train],axis=1)
df_test=pd.concat([x_test,y_test],axis=1)

for i in [df_train,df_test,test]:
    i.drop(['Name','Ticket'],axis=1,inplace=True)

In [225]:
categorical_features=[i for i in df_train.columns if df_train[i].dtype=='O']
numerical_features=[i for i in df_train.columns if df_train[i].dtype!='O']
print('categorical_features :\n',categorical_features,'\nnumerical_features :\n',numerical_features)

categorical_features :
 ['Sex', 'Cabin', 'Embarked', 'Title'] 
numerical_features :
 ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [226]:
#irq removendo outliers
for feature in numerical_features:
    if 0 in df_train[feature].unique():
        pass
    else:
        q1=df_train[feature].quantile(0.25)
        q3=df_train[feature].quantile(0.75)
        iqr=q3-q1
        lower_limit=q1-1.5*iqr
        upper_limit=q3+1.5*iqr
        df_train=df_train[(df_train[feature]<upper_limit) & (df_train[feature]>lower_limit)]

In [227]:
#tratando dados nulos
#preenchendo valores ausentes em 'Age' com a mediana
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_test['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)

#preenchendo valores ausentes em 'Embarked' com o valor mais frequente
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Embarked'].fillna(df_test['Embarked'].mode()[0], inplace=True)
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

#preenchendo valores ausentes em 'Fare' com a mediana
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

for df in [df_train, df_test, test]:
    df['Cabin'].fillna('Unknown', inplace=True)
    df['Cabin'] = df['Cabin'].astype(str)

print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(test.isnull().sum())

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
Survived       0
dtype: int64
PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64


In [228]:
#usando lable encoder
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()

    le.fit(df_train[feature].unique().tolist() + df_test[feature].unique().tolist() + test[feature].unique().tolist())
    df_train[feature] = le.transform(df_train[feature])
    df_test[feature] = le.transform(df_test[feature])
    test[feature] = le.transform(test[feature])
    label_encoders[feature] = le

In [229]:
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']
X_test = df_test.drop('Survived', axis=1)
y_test = df_test['Survived']

In [230]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [231]:
y_pred = model.predict(X_test)

In [236]:
#avaliacao do modelo
accuracy= accuracy_score(y_test, y_pred)
print("Acurácia:", accuracy)

cm= confusion_matrix(y_test, y_pred)

tn, fp, fn, tp= cm.ravel()

precision= tp/(tp+fp)
recall= tp/(tp+fn)
f1_score= 2 *(precision*recall)/(precision+recall)
specificity= tn/(tn+fp)

print("\nPrecisão:", precision)
print("Especificidade:", specificity)
print("Recall:", recall)
print("F1-score:", f1_score)

Acurácia: 0.7052238805970149

Precisão: 0.5981308411214953
Especificidade: 0.7440476190476191
Recall: 0.64
F1-score: 0.6183574879227053


In [235]:
test_predictions = model.predict(test)

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_predictions
})

submission.to_csv('titanicDtree.csv', index=False)